In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
weibo_m = pd.read_csv('./data/Weibo_M.csv')
weibo = pd.read_csv('./data/Weibo.csv')
wechat = pd.read_csv('./data/Wechat.csv')
news_forum = pd.read_csv('./data/News_forum.csv')

In [3]:
weibo_m = weibo_m[['COMMENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
weibo = weibo[['COMMENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
wechat = wechat[['CONTENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')
news_forum = news_forum[['CONTENT','AREAS AFFECTED']].dropna(axis=0, how = 'any')

In [4]:
weibo_m.columns = ['CONTENT','AREAS AFFECTED']
weibo.columns = ['CONTENT','AREAS AFFECTED']

In [9]:
data = pd.concat([weibo_m,weibo,wechat,news_forum])

In [10]:
data['CONTENT'] = data['CONTENT'].str.replace('\n', '')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('life', 'Life')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('sponsored', 'Sponsored')
data['AREAS AFFECTED'] = data['AREAS AFFECTED'].str.replace('news', 'News')

In [11]:
data['AREAS AFFECTED'].unique()

array(['Life Comprehend', 'Products/Service', 'Health Information',
       'Ads/Marketing', 'Customer Service', 'Irrelevant Ads',
       'Charity Events', 'Survey/Questions', 'Contact Information', 'Fund',
       'Corporate Brands', 'Sponsored Events', 'Volunteering Activity',
       'Products', 'Website Issues', 'Corporate News', 'Recruitment',
       'Agent Issues', 'Employment', 'Stocks&Earnings',
       'General Mentioned', 'Employee Information', 'General Info', 'Agent'], dtype=object)

In [12]:
data.shape

(30894, 2)

In [18]:
for x in data['AREAS AFFECTED'].unique():
    np.savetxt('./data/processed/'+ x.replace('/',' ') +'.utf8',\
                   data[data['AREAS AFFECTED'] == x]['CONTENT'].values, fmt='%s')

In [19]:
from os import listdir
from os.path import isfile, join
import data_helpers
mypath = './data/processed/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]


In [31]:
for i in range(len(onlyfiles)):
    if i == 0:
        x_text = data_helpers.read_and_clean_zh_file(mypath+onlyfiles[i])
    else:
        x_text += data_helpers.read_and_clean_zh_file(mypath+onlyfiles[i])
    

In [32]:
len(x_text)

30894

In [ ]:
data_helpers.read_and_clean_zh_file